# Opening a New Movie Theater in Tokyo

# 1. Introduction

With a population of over 37 million people, Tokyo, is the most populated metropolitan area in the world. Modern, stylish and full of culture, the city of Tokyo provides many opportunity for entrepreneurs to open new businesses. 

The cinema industry has been incredibly resilient for more than a century, and according to the Financial Times, in 2019, global cinema box office reached a new high of over $42bn. Over the last decades, the global industry sales have steadily increased. However, the main driver of this is likely the steady increase in ticket prices, rather than an increase in attendance. In fact, if we look at the US, the number of tickets purchased in 2019 was roughly 1.26 billion, which is the lowest attendance since 2011. It is however worth considering that while attendace is dropping in North America, the numbers overseas continue to rise, especially in Asia. In addition, certain experiences such as those provided by IMAX, cannot be recreated at home. 

### 1.1. Business Problem

The objective of this project is to find the best location to open a new movie theater. In order to find the best location, we'll look at the areas with the lowest concentration of cinemas but a high concentration of what we can define as "complementary" venues. These are places that are generally visited in conjunction with movie theathers.

Restaurants can undoubtedly be considered as such "complementary" venues. People love to go out and take a bite before going to watch the latest movie at the cinema, and for this reason many restaurants often offer menu deals with tickets to the movie theather next door. 

For the purpose of this project, we'll be looking at areas with a high concentration of Italian Restaurants. Italian food, or its fusion with Japanese food, Itameshi, has been one of the most loved cuisines by Japanese people since the 1990s. 

### 1.2. Target Audience

This analysis can be useful for:
    1. Entrepreneurs looking to open a new business
    2. Local cinema companies looking to expand in a new area
    3. Foreign cinema companies looking to expand in a new territory with little competition

## 2. Data Requirements and Sources

### 2.1 Data Requirements

To complete the analysis, we require the following data:
    1. List of neighborhoods in Tokyo
    2. Geographical coordinates of the neighborhoods to plot the data on a map
    3. Tokyo movie theathers data
    4. Tokyo Italian restaurants data

### 2.2 Data Sources

The list of neighborhoods in Tokyo will be extracted from [this](https://en.wikipedia.org/wiki/Category:Neighborhoods_of_Tokyo) Wikipedia page.
The latitude and longitude coordinates of the neighborhoods will be retrieved using the Geocoder package. 
The data for movie theathers and Italian restaurants will be retrieved using the FourSquare API.

## 3. Metholology

In [1]:
# import libraries
import numpy as np 
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
!pip install geocoder 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
!pip install folium
import folium # map rendering library

print("Libraries imported.")

  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491061 sha256=61b3a0db8e1a6d9ad994f947c438e285762b5271e3b88fa498f76e6d5ce03518
  Stored in directory: c:\users\tanma\appdata\local\pip\cache\wheels\6e\9c\ed\4499c9865ac1002697793e0ae05ba6be33553d098f3347fb94
Successfully built future


You should consider upgrading via the 'c:\users\tanma\appdata\local\programs\python\python36\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\tanma\appdata\local\programs\python\python36\python.exe -m pip install --upgrade pip' command.


Libraries imported.


### 3.1 Data Gathering and Preparation

In [2]:
# Retrieve data from Wikipedia page
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighborhoods_of_Tokyo").text

# Parse data
soup = BeautifulSoup(data, 'html.parser')

# Create empty neighborhood list
neighborhoodList = []

for row in soup.find_all("div", class_="mw-content-ltr")[0].findAll("li"):
    neighborhoodList.append(row.text)
df = pd.DataFrame({"Neighborhood": neighborhoodList})
df.head()

,Neighborhood
0,"► Akasaka, Tokyo‎ (16 P)"
1,"► Akihabara‎ (1 C, 17 P)"
2,► Asakusa‎ (12 P)
3,► Harajuku‎ (10 P)
4,► Ikebukuro‎ (11 P)


In [3]:
# Create new dataframe with only the neighborhoods and removing irrelevant subcategories
tokyo_df = df.iloc[12:].reset_index(drop=True)
tokyo_df.head()

,Neighborhood
0,Agariyashiki
1,Akihabara
2,"Aoyama, Minato, Tokyo"
3,Asagaya
4,Banchō


In [4]:
tokyo_df.shape

(84, 1)

In [5]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Tokyo, Japan'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [6]:
coordinates = [ get_latlng(neighborhood) for neighborhood in tokyo_df["Neighborhood"].tolist() ]

In [7]:
#create df to put the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coordinates, columns=['Latitude', 'Longitude'])
    
# merge the coordinates into the main dataframe
tokyo_df['Latitude'] = df_coords['Latitude']
tokyo_df['Longitude'] = df_coords['Longitude']

# check the neighborhoods and the coordinates
print(tokyo_df.shape)
tokyo_df.head()

(84, 3)


,Neighborhood,Latitude,Longitude
0,Agariyashiki,35.726462,139.705156
1,Akihabara,35.702171,139.774409
2,"Aoyama, Minato, Tokyo",35.658017,139.751546
3,Asagaya,35.704890,139.636260
4,Banchō,35.691973,139.741446


In [8]:
# get the coordinates of Tokyo
address = 'Tokyo, Japan'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Tokyo are: {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Tokyo are: 35.6828387, 139.7594549.


In [9]:
# create map of Tokyo using latitude and longitude values
tokyo_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(tokyo_df['Latitude'], tokyo_df['Longitude'], tokyo_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(tokyo_map)  
    
tokyo_map

### 3.2 Using the FourSquare API

In [12]:
# Foursquare credentials and version
CLIENT_ID = 'O4KNNRFK0AAZXWP4DBZPEDQPQ2PYL0AYLAY2SOAFZCIPR352' # your Foursquare ID
CLIENT_SECRET = 'BYQ4EVBZ4EUIGNIP5ZEQIWASGZ2JWU5NLH0RPZ44LYDE0IHX' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: O4KNNRFK0AAZXWP4DBZPEDQPQ2PYL0AYLAY2SOAFZCIPR352
CLIENT_SECRET:BYQ4EVBZ4EUIGNIP5ZEQIWASGZ2JWU5NLH0RPZ44LYDE0IHX


In [13]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(tokyo_df['Latitude'], tokyo_df['Longitude'], tokyo_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [14]:
# convert the venues list into a new dataframe
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(8369, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Agariyashiki,35.726462,139.705156,Jiyu Gakuen Myonichikan (自由学園明日館),35.726474,139.707315,Event Space
1,Agariyashiki,35.726462,139.705156,Meijiro Garden (目白庭園),35.724543,139.705781,Garden
2,Agariyashiki,35.726462,139.705156,Kitchen ABC (キッチンABC),35.730416,139.707433,Yoshoku Restaurant
3,Agariyashiki,35.726462,139.705156,Mejiro Tanakaya (目白田中屋),35.721736,139.706342,Liquor Store
4,Agariyashiki,35.726462,139.705156,Aigre Douce (エーグル ドゥース),35.722327,139.700745,Dessert Shop


In [15]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 292 uniques categories.


In [16]:
# print a list of 50 unique categories
venues_df['VenueCategory'].unique()[:50]

array(['Event Space', 'Garden', 'Yoshoku Restaurant', 'Liquor Store',
       'Dessert Shop', 'Café', 'Vietnamese Restaurant',
       'Sporting Goods Shop', 'Sushi Restaurant', 'Theater',
       'Szechuan Restaurant', 'Wagashi Place', 'Bookstore', 'Sake Bar',
       'Ramen Restaurant', 'Chinese Restaurant', 'Comedy Club', 'Plaza',
       'Udon Restaurant', 'Pub', 'French Restaurant', 'Coffee Shop',
       'Pastry Shop', 'Soup Place', 'German Restaurant', 'Pie Shop',
       'Soba Restaurant', 'Pet Café', 'Asian Restaurant', 'Steakhouse',
       'Used Bookstore', 'Hotpot Restaurant', 'Japanese Restaurant',
       'Park', 'Yakitori Restaurant', 'Dim Sum Restaurant',
       'Movie Theater', 'Grocery Store', 'Beer Bar', 'Clothing Store',
       'Noodle House', 'Gym / Fitness Center', 'Pizza Place',
       'Japanese Curry Restaurant', 'BBQ Joint', 'Juice Bar', 'Spa',
       'Nightclub', 'Hobby Shop', 'Bubble Tea Shop'], dtype=object)

In [17]:
# one hot encoding
tokyo_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tokyo_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tokyo_onehot.columns[-1]] + list(tokyo_onehot.columns[:-1])
tokyo_onehot = tokyo_onehot[fixed_columns]

print(tokyo_onehot.shape)
tokyo_onehot.head()

(8369, 293)


,Neighborhoods,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Baseball Stadium,Bath House,Bay,Beach,Bed & Breakfast,Beer Bar,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buddhist Temple,Buffet,Burger Joint,Burmese Restaurant,Bus Stop,Butcher,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Castle,Cemetery,Cha Chaan Teng,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Czech Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donburi Restaurant,Dongbei Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Fishing Spot,Fishing Store,Flower Shop,Food & Drink Shop,Food Court,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Heliport,Herbs & Spices Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kaiseki Restaurant,Karaoke Bar,Karaoke Box,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Kushikatsu Restaurant,Laboratory,Lake,Library,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Mongolian Restaurant,Monjayaki Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nabe Restaurant,Nightclub,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant,Opera House,Optical Shop,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Port,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Resort,Rest Area,Restaurant,River,Road,Rock Club,Rugby Stadium,Russian Restaurant,Sake Bar,Salad Place,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shrine,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Strip Club,Sukiyaki Restaurant,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Takoyaki Place,Tapas Restaurant,Tea Room,Tempura Restaurant,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tonkatsu Restaurant,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Tree,Tunnel,Turkish Restaurant,Udon Restaurant,Unagi Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,

In [18]:
tokyo_grouped = tokyo_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(tokyo_grouped.shape)
tokyo_grouped

(84, 293)


,Neighborhoods,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Baseball Stadium,Bath House,Bay,Beach,Bed & Breakfast,Beer Bar,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buddhist Temple,Buffet,Burger Joint,Burmese Restaurant,Bus Stop,Butcher,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Castle,Cemetery,Cha Chaan Teng,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Czech Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donburi Restaurant,Dongbei Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Fishing Spot,Fishing Store,Flower Shop,Food & Drink Shop,Food Court,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Heliport,Herbs & Spices Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kaiseki Restaurant,Karaoke Bar,Karaoke Box,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Kushikatsu Restaurant,Laboratory,Lake,Library,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Mongolian Restaurant,Monjayaki Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nabe Restaurant,Nightclub,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant,Opera House,Optical Shop,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Port,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Resort,Rest Area,Restaurant,River,Road,Rock Club,Rugby Stadium,Russian Restaurant,Sake Bar,Salad Place,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shrine,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Strip Club,Sukiyaki Restaurant,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Takoyaki Place,Tapas Restaurant,Tea Room,Tempura Restaurant,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tonkatsu Restaurant,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Tree,Tunnel,Turkish Restaurant,Udon Restaurant,Unagi Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,

In [19]:
# Check how many movie theaters there are in the data gathered
len(tokyo_grouped[tokyo_grouped["Movie Theater"] > 0])

41

In [20]:
# Check how many Italian restaurants there are in the data gathered
len(tokyo_grouped[tokyo_grouped["Italian Restaurant"] > 0])

68

In [21]:
# Build a new dataframe with the concentration of movie theathers and Italian restaurants per neighborhood
new_df = tokyo_grouped[["Neighborhoods","Movie Theater","Italian Restaurant" ]]
new_df.head()

,Neighborhoods,Movie Theater,Italian Restaurant
0,Agariyashiki,0.01,0.00
1,Akihabara,0.00,0.01
2,"Aoyama, Minato, Tokyo",0.00,0.03
3,Asagaya,0.01,0.05
4,Banchō,0.01,0.09


### 3.3 K-Means Clustering

In [22]:
# set number of clusters
kclusters = 3

tokyo_clustering = new_df.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tokyo_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 1, 1, 2, 0, 2, 2, 1, 0])

In [23]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
tokyo_merged = new_df.copy()

# add clustering labels
tokyo_merged["Cluster Labels"] = kmeans.labels_

In [24]:
tokyo_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
tokyo_merged.head()

,Neighborhood,Movie Theater,Italian Restaurant,Cluster Labels
0,Agariyashiki,0.01,0.00,0
1,Akihabara,0.00,0.01,0
2,"Aoyama, Minato, Tokyo",0.00,0.03,1
3,Asagaya,0.01,0.05,1
4,Banchō,0.01,0.09,2


In [25]:
#merge dataframes to add latitude/longitude for each neighborhood
tokyo_merged = tokyo_merged.join(tokyo_df.set_index("Neighborhood"), on="Neighborhood")

print(tokyo_merged.shape)
tokyo_merged.head() 

(84, 6)


,Neighborhood,Movie Theater,Italian Restaurant,Cluster Labels,Latitude,Longitude
0,Agariyashiki,0.01,0.00,0,35.726462,139.705156
1,Akihabara,0.00,0.01,0,35.702171,139.774409
2,"Aoyama, Minato, Tokyo",0.00,0.03,1,35.658017,139.751546
3,Asagaya,0.01,0.05,1,35.704890,139.636260
4,Banchō,0.01,0.09,2,35.691973,139.741446


In [26]:
# sort the results by Cluster Labels
print(tokyo_merged.shape)
tokyo_merged.sort_values(["Cluster Labels"], inplace=True)
tokyo_merged.head()

(84, 6)


,Neighborhood,Movie Theater,Italian Restaurant,Cluster Labels,Latitude,Longitude
0,Agariyashiki,0.01,0.0,0,35.726462,139.705156
57,Shinjuku Ni-chōme,0.01,0.0,0,35.693798,139.703440
56,Shinjuku Golden Gai,0.01,0.0,0,35.693798,139.703440
54,Shin-Kiba,0.00,0.0,0,35.633930,139.832456
53,Shimokitazawa,0.00,0.0,0,35.661440,139.667340


## 4. Results

In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tokyo_merged['Latitude'], tokyo_merged['Longitude'], tokyo_merged['Neighborhood'], tokyo_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 3.5 View of Individual Clusters

In [28]:
tokyo_merged.loc[tokyo_merged['Cluster Labels'] == 0]

,Neighborhood,Movie Theater,Italian Restaurant,Cluster Labels,Latitude,Longitude
0,Agariyashiki,0.01,0.000000,0,35.726462,139.705156
57,Shinjuku Ni-chōme,0.01,0.000000,0,35.693798,139.703440
56,Shinjuku Golden Gai,0.01,0.000000,0,35.693798,139.703440
54,Shin-Kiba,0.00,0.000000,0,35.633930,139.832456
53,Shimokitazawa,0.00,0.000000,0,35.661440,139.667340
51,Sendagaya,0.01,0.020000,0,35.678931,139.707465
49,San'ya,0.01,0.020000,0,35.689456,139.691716
58,Shinjuku Southern Terrace,0.01,0.010000,0,35.687517,139.700400
48,Otome Road,0.01,0.020000,0,35.689456,139.691716
45,Ochanomizu,0.00,0.000000,0,35.700696,139.763721


In [29]:
tokyo_merged.loc[tokyo_merged['Cluster Labels'] == 1]

,Neighborhood,Movie Theater,Italian Restaurant,Cluster Labels,Latitude,Longitude
8,Ebisunishi,0.01,0.04,1,35.649160,139.706685
74,Yotsuya,0.00,0.03,1,35.687818,139.719711
69,"Uehara, Shibuya",0.00,0.04,1,35.664739,139.680026
68,Toyosu,0.00,0.04,1,35.655406,139.792671
75,Yoyogi,0.01,0.03,1,35.682619,139.694331
3,Asagaya,0.01,0.05,1,35.704890,139.636260
66,"Tateishi, Tokyo",0.00,0.04,1,35.740863,139.846903
2,"Aoyama, Minato, Tokyo",0.00,0.03,1,35.658017,139.751546
17,"Honmachi, Shibuya",0.00,0.05,1,35.685960,139.680318
36,Motoyoyogichō,0.00,0.04,1,35.672117,139.685543


In [30]:
tokyo_merged.loc[tokyo_merged['Cluster Labels'] == 2]

,Neighborhood,Movie Theater,Italian Restaurant,Cluster Labels,Latitude,Longitude
10,Hatsudai,0.00,0.06,2,35.676543,139.685445
7,"Ebisuminami, Shibuya",0.01,0.06,2,35.643475,139.708589
14,"Hiroo, Shibuya",0.02,0.06,2,35.651373,139.714790
24,Kagurazaka,0.01,0.08,2,35.702125,139.738778
6,"Ebisu, Shibuya",0.02,0.07,2,35.644429,139.718256
4,Banchō,0.01,0.09,2,35.691973,139.741446
18,Ichigaya,0.01,0.08,2,35.692406,139.735843
34,"Mita, Meguro, Tokyo",0.02,0.08,2,35.640176,139.710689


## 5. Discussion

We have successfully divided Tokyo neighborhoods into 3 different clusters. We can see that cluster 2 (purple color on the map) is the smallest one and has the highest concentration of movie theathers and Italian restaurants. Intead, Cluster 0 (red color on the map) is the biggest cluster and has the lowest concentration of such venues. Cluster 1 (green color on the map) contains a reasonable list of neighborhoods with a relatively significant number of Italian restaurants, but with little movie theaters. In fact, the neighborhoods in cluster 1 represent the best choice for areas to open a new movie theater.

Looking at the neighboorhoods in cluster 2, we can see that there is one neighborhood with a high concentration of Italian restaurants but no movie theathers, this is Hatshudai. With a quick search on Google, one can see that Hatsudai is a residential area, home to the Opera City tower and the New National Theatre. It therefore sounds like the perfect place to open a new movie theatre.

## 6. Conclusion

In this research, we have gone through the full journey of identifying a business problem, gathering the data required to perform the analysis, preparing the data to be ready for modelling, performed machine learning clustering technique on the data to divide the neighborhoods in 3 groups based on their similarities, and finally analysing the results to find the solution to the initial problem.

The goal of this research was to find suitable areas to open a new movie theater, prioritising areas where there are little no to movie theaters, but with a high concentration of Italian restaurants. 

The analysis suggests that the neighborhoods in cluster 1 are all suitable areas to open a new movie theater. However, the best neihghborhood for our purpose is probably Hatsudai, in cluster 2, currently with no cinemas but with various Italian restaurants. 